### In this hands on you will build a model that once trained on a peice of text data can generate its own sequnce of words in a similar fashion as in trained data.

- Follow the instructions provided for each cell and and code accordingly. 
- In order to run the cell press shift+enter.
- make sure you have run all the cells before submitting the hands on

### Run the below cell to import necessary packages

In [1]:
import numpy as np
import pandas as pd
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


### Read the text data from story.txt file and split the text into seperate tokens, assign thr array of tokens to variable *training_data*

### Expected output:
    array(['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general',
       'council', 'to']

In [56]:
### Start code here



training_data = open("story.txt").read().split(" ")
###End code
training_data[:10]

['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general', 'council', 'to']

### Take the unique tokens in training_data nas sort them alphabetical order and assign the sorted list to variable words
### create dictionary ind_to_word to map index to word
### create another dictionary word_to_ind to reverse map word to their respective index.

In [57]:
####Start code here
words = sorted(set(training_data))
ind_to_word = {i:w for i, w in enumerate(words)}
word_to_ind = {w:i for i, w in enumerate(words)}
###End code
print("words: ", words[:10], "\n")
print("index_to_words: ", list(ind_to_word.items())[:10], "\n")
print("word_to_index: ", list(word_to_ind.items())[:10], "\n")

words:  ['', ',', '.', '.\n', '?', 'a', 'about', 'ago', 'agree', 'all'] 

index_to_words:  [(0, ''), (1, ','), (2, '.'), (3, '.\n'), (4, '?'), (5, 'a'), (6, 'about'), (7, 'ago'), (8, 'agree'), (9, 'all')] 

word_to_index:  [('', 0), ('remedies', 73), ('approach', 15), ('?', 4), ('until', 96), ('neighbourhood', 61), ('long', 50), ('to', 94), ('is', 46), ('of', 64)] 



### write a function to generate training dataset 
    - parameters: dataset: orginal dataset
                  look_back: the window size that tells the number of previous values in the series to look for to                   predict the next one.
    - returns: feature and target arrays
    
example: 
         for window size 1:
         dataset = [1,2,3,4]  
         feature = [[1],[2],[3]]    
         target = [2,3,4]  
         
         for window size 2:
         feature = [[1,2],[2,3]]  
         target = [3,4]  
### Expected output when you when you call generate_dataset on training_data and look_back = 10 :
input:  [[48, 5, 0, 85, 56, 37, 3, 35, 28, 92], [5, 0, 85, 56, 37, 3, 35, 28, 92, 25], [0, 85, 56, 37, 3, 35, 28, 92, 25, 102]]  
labels:  [25, 102, 53]


In [58]:
####Start code here
def generate_dataset(dataset, look_back=10):

    dataX, dataY = [], []
    for i in range(len(dataset)-look_back - 1):
       
        seq_in = dataset[i:i + look_back]
        #print('seq in : ', seq_in)
        seq_out = dataset[i + look_back]
        #print('seq out : ', seq_out)
        dataX.append([word_to_ind[char] for char in seq_in])
        dataY.append(word_to_ind[seq_out])
    return dataX, dataY

    
###End code

inputs, labels = generate_dataset(training_data, 10)
print("input: ", inputs[:3])
print("labels: ", labels[:3])

input:  [[50, 7, 1, 87, 58, 39, 5, 37, 30, 94], [7, 1, 87, 58, 39, 5, 37, 30, 94, 27], [1, 87, 58, 39, 5, 37, 30, 94, 27, 104]]
labels:  [27, 104, 55]


### Next step is to  reshape the inputs and normalize them.
    - This step is coded for you
    - Run the below cell to prepare the data for training the model

In [59]:
look_back = 10
X_modified = np.reshape(inputs, (len(inputs), look_back, 1))
X_modified = X_modified / float(len(words))
Y_modified = np_utils.to_categorical(labels)

### Using keras Sequential() class create a model having two LSTM block and one fully connected  layer with activation softmax
### apply dropout with probability p = 0.2 between the layers of LSTM
### compile the model with categorical_crossentropy loss and  adam optimizer

### Expected output
<img src="lstm.png">

In [70]:
np.random.seed(51)
###Start code here
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))

model.add(Dense(Y_modified.shape[1], activation='softmax'))


###End code
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

(194, 114)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 10, 400)           643200    
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 400)           0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 400)               1281600   
_________________________________________________________________
dropout_9 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 114)               45714     
Total params: 1,970,514
Trainable params: 1,970,514
Non-trainable params: 0
_________________________________________________________________


### Run model.fit() on train data with features as X_modified and target as Y_modified for 50 epoches and batch_size = 10

In [71]:
###Start code here
train_logs = model.fit(X_modified, Y_modified, epochs = 50, batch_size = 10) 
###End code
with open("output.txt", "w+") as file:
    file.write("train score {0:.2f}\n".format(train_logs.history["loss"][-1]))

Epoch 1/50
194/194 [==============================] - 5s 28ms/step - loss: 4.7290
Epoch 2/50
194/194 [==============================] - 4s 20ms/step - loss: 4.5844
Epoch 3/50
194/194 [==============================] - 4s 20ms/step - loss: 4.5346
Epoch 4/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4851
Epoch 5/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4801
Epoch 6/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4514
Epoch 7/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4505
Epoch 8/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4639
Epoch 9/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4636
Epoch 10/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4328
Epoch 11/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4291
Epoch 12/50
194/194 [==============================] - 4s 20ms/step - loss: 4.4422
Epoch 13/50
1

### The below codes takes a random sequence of words and generates more sequnce using the model you trained above.

In [72]:
string_mapped = inputs[50].copy()
full_string = [ind_to_word[value] for value in string_mapped]
# generating characters
for i in range(100):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(words))
    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [ind_to_word[value] for value in string_mapped]
    full_string.append(ind_to_word[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

    
txt=""
for word in full_string:
    txt = txt+" "+word
print(txt)

 which he thought would meet the case . you will met agree some up bell to this easily , case an old we we our up danger cat . this propose met with said always know had mice propose means we we up is had you their the agree met with , cat . agree , should very , this danger the the cat it the case bell all remedies looked should very , , of it but and we always this danger what the the would small outwit easy will cat proposal met met with , and , looked remedies got we always know danger the  but make
